**Navn på gruppens medlemmer:** Tore B. Braathen, Åshild M. M. Føllesdal, Yawar Mahmood

*Levér en ferdig kjørt notebook som inkluderer alle resultater, helst eksportert som HTML-dokument. Det er tilstrekkelig at ett gruppemedlem leverer på vegne av alle i gruppa.*

# Tidsutvikling av bølgefunksjonen

Utviklingen av en tilstand $\Psi = \Psi(x, t)$ over tid er beskrevet av den tidsavhengige Schrödingerlikningen
$$ i \hbar \frac{\partial \Psi}{\partial t} = \hat{H} \Psi = -\frac{\hbar^2}{2m} \frac{\partial^2 \Psi}{\partial x^2} + V \Psi $$

Ved hjelp av separasjon av variable er det mulig å vise at en generell løsning av denne ligningen er
$$ \Psi(x, t) = \sum_n c_n \psi_n(x) e^{-i E_n t / \hbar} $$
der summen går over alle de stasjonære tilstandene $\psi_n$ med energier $E_n$ som løser den tidsuavhengige Schrödingerligningen
$$ \hat{H} \psi_n = -\frac{\hbar^2}{2 m} \psi_n'' + V \psi_n = E_n \psi_n $$

Det viser seg at *enhver* tilstand $\Psi(x, t)$ kan skrives som en slik lineærkombinasjon av de stasjonære tilstandene, og vi sier derfor at de utgjør et *fullstendig sett*.
Gitt en vilkårlig starttilstand $\Psi_0 = \Psi(x, 0)$, er det bare snakk om å finne de rette konstantene $c_n$ som oppfyller
$$ \Psi_0 = \sum_n c_n \psi_n $$
og vi har da all informasjonen vi trenger for å beregne $\Psi$ for alle $t$.
Dette er ingen vanskelig oppgave, da de stasjonære tilstandene er *ortogonale*, og *ortonormale*, så lenge vi velger dem normerte, dvs.
$$\int \psi_m \psi_n \mathrm{d}x = \delta_{m n} = \begin{cases}0 & \text{for}\,\, m \neq n \\ 1 & \text{for}\,\, m = n\end{cases}$$
Vi kan utnytte dette til å "plukke ut" en av konstantene $c_m$ i lineærkombinasjonen ved å integrere begge sider av likningen:
$$ \int \Psi_0 \psi_m \mathrm{d}x = \int \sum_n c_n \psi_n \psi_m \mathrm{d}x = \sum_n c_n \int \psi_n \psi_m \mathrm{d}x = \sum_n c_n \delta_{m n} = c_m $$

I denne øvingen skal vi se nærmere på egenskapene om ortogonalitet og fullstendighet i en diskretisert modell og utnytte dette til å studere tidsutviklingen til vilkårlige starttilstander i vilkårlige potensialer.

Vi bruker samme modell for diskretiseringen av rommet som i den første numeriske øvingen.
Der diskretiserte vi rommet i $N + 2$ ekvidistante punkter fra $x_0$ til $x_{N+1}$ og satte $V(x \leq x_0) = V(x \geq x_{N+1}) = \infty$, slik at $\psi_n(x \leq x_0) = \psi_n(x \geq x_{N+1}) = 0$.
Vi innførte så et vilkårlig potensial $\boldsymbol{V} = [V(x_1), \ldots, V(x_N)]^T$ på rutenettet $\boldsymbol{x} = [x_1, \ldots, x_N]^T$ og fant de stasjonære tilstandene $\boldsymbol{\psi_n} = [\psi_n(x_1), \ldots, \psi_n(x_N)]^T$ og energiene $E_n$ ved å finne egenverdiene og egenvektorene til Hamiltonmatrisen $H$.

Hamiltonmatrisen $H$ er en reell og symmetrisk matrise, og har da ifølge et teorem fra lineæralgebraen ortogonale egenvektorer.
Vi kan så oversette formen for den generelle løsningen på den tidsavhengige Schrödingerlikningen til vår diskretiserte modell med vektorlikningen
$$ \boldsymbol{\Psi} = \boldsymbol{\Psi}(x, t) = [\Psi(x_1, t), \ldots, \Psi(x_N, t)]^T = \sum_{n=1}^{N} c_n \boldsymbol{\psi_n} e^{-i E_n t / \hbar} $$
I denne diskretiserte modellen får vi dermed helt tilsvarende ortogonalitets- og fullstendighetsegenskaper som med de eksakte funksjonene $\psi$ og $\Psi$.

In [ ]:
# uncomment ONE line to choose matplotlib backend
# if using Jupyter Notebook, use interactive "notebook" backend for best results
# if using Jupyter Lab, use interactive "widget" backend for best results
# if both fail, use static "inline" backend
%matplotlib notebook 
#%matplotlib widget 
#%matplotlib inline 


# importing necessary libraries
import matplotlib.pyplot as plt
import matplotlib.animation
import numpy as np
from IPython.display import HTML, clear_output


# helpful to raise exception upon complex number calculation error
# instead of just a not descriptive warning
import warnings
warnings.filterwarnings(action="error", category=np.ComplexWarning)


For å finne de stasjonære tilstandene $\boldsymbol{\psi_n}$ og energiene $E_n$ til en partikkel i et vilkårlig potensial $\boldsymbol{V}$, bruker vi teknikken fra den første numeriske øvingen.
Vi har lagt ved fungerende funksjonalitet for dette i funksjonen `get_stationary_states()` under. 
**Merk at**
* **det brukes atomære enheter med $\hbar = m_e = 1$, slik at naturlige avstander og tider også har tallverdier i størrelsesorden rundt 1,**
* **de stasjonære tilstandene $\boldsymbol{\psi_n}$ normeres i forstanden $\boldsymbol{\psi_m} \cdot \boldsymbol{\psi_n} = \sum_{i=1}^N \psi_{m,i} \psi_{n,i} \Delta x = \delta_{mn}$ (der $\psi_{n,i}$ er $i$-te element i vektoren $\boldsymbol{\psi_n}$), i tråd med normeringsintegralet $\int \psi_m \psi_n \mathrm{d}x \approx \sum_{i=1}^N \psi_m(x_i) \psi_n(x_i) \Delta x$, og at**
* **de stasjonære tilstandene kommer ut i en $N \times N$ matrise med $\boldsymbol{\psi_n}$ langs den $n$-te *raden*, som så kan hentes ut med `waves[n-1]`.**

Du kan selvfølgelig erstatte dette med din egen løsning fra den første numeriske øvingen, da du kanskje er bedre kjent med denne. For å passe inn med noe inkludert programkode senere i øvingen, anbefaler vi likevel at kravene over er tilfredsstilt.

In [ ]:
# Defining constants in SI units
hb: float = 1.05e-34 #hbar - plancks constant/2*pi
me: float = 9.1e-31 #electron mass
e: float = 1.6e-19 #electron charge
a0: float = 5.29e-11 #adjustment constant - go from atomic units to SI units 
Eh: float = hb**2/(me*a0**2) #Energy
atomic_units: bool = False

In [ ]:
# Defining constants in atomic units
hb: float = 1.
me: float = 1.
e: float = 1.
a0: float = 1.
Eh: float = hb**2/(me*a0**2)
atomic_units: bool = True

In [ ]:
def hamiltonian(x: np.ndarray, V: np.ndarray, m: float = me) -> np.ndarray:  # function returns an 2D-array, Hamilton-matrix
    dx: float = x[1] - x[0]  # defines step length dx
    diag: np.ndarray = V + hb**2/(m * dx**2)  # defines the diagonal of the matrix by given formula
    semidiag: np.ndarray = np.full(len(x)-1, -hb**2/(2 * m * dx**2))  # defines the semi-diagonal of the matrix by given formula
    return np.diag(diag) + np.diag(semidiag, k=1) + np.diag(semidiag, k=-1)


def normalize(vecs: np.ndarray, x: np.ndarray) -> np.ndarray:  # function returns an 1D-array, of normalized values vecs
    dx: float = x[1] - x[0]  # defines step length dx 
    normalizing_values = np.sqrt(np.sum(np.abs(vecs)**2 * dx, axis=1)[np.newaxis].T)
    return vecs / normalizing_values


def get_stationary_states(x: np.ndarray, V: np.ndarray, m: float = me):  # returns a vector of energies, and a 2D-matrix of corresponding wave functions
    H: np.ndarray = hamiltonian(x, V, m) #make Hamilton-matrix
    energies, eigenvectors = np.linalg.eigh(H) #find energies and eigenvectors
    waves: np.ndarray = normalize(eigenvectors.T, x) #find wave functions using eigenvectors
    # waves[waves[:, 1] < 0.] *= -1  # Can be used to ensure that all wave functions start at a positive value
    return energies, waves


Nedenfor har vi konstruert noen potensialer $\boldsymbol{V}$.

**For hvert av disse potensialene, finn de stasjonære tilstandene for et elektron og bekreft at de er ortonormale og utgjør et fullstendig sett.**

**Ta deg friheten til å endre litt på potensialene, om de ikke "passer inn" i programmet ditt i formen under.**

In [ ]:
m = me  # the mass of the particle
w = 1. * Eh/hb
x, dx = np.linspace(-20*a0, +20*a0, 400, retstep=True)  # defines list x and step length dx


# predefined potentials, with their names given in the list titles
v_infwell = np.zeros_like(x)
v_harmosc = 0.5*m*w**2*x**2
v_finwell = np.piecewise(x, [np.abs(x) > 2*a0, np.abs(x) <= 2*a0], [0.*Eh, 40.*Eh])
v_weird = np.random.rand(len(x))
potentials = [v_infwell, v_harmosc, v_finwell, v_weird]
titles = ["Infinite well", "Harmonic oscillator", "Finite well", "Weird potential"]


def orthonormality(psi: np.ndarray, dx: float) -> np.ndarray:
    """Returns a 2D grid where grid[i,j] = psi[i] \\dot psi[j] * dx"""
    return psi @ psi.T * dx
# for more info on the usage of @, check: https://numpy.org/doc/stable/reference/generated/numpy.dot.html
# introduced in PEP 465: https://www.python.org/dev/peps/pep-0465/


# calculation of various properties and visualization of results
fig = plt.figure(figsize=(10, 10))
for i in range(len(potentials)):
    energies, waves = get_stationary_states(x, potentials[i], m)

    orthonormal_grid = orthonormality(waves, dx)

    ax = fig.add_subplot(2, 2, i+1)
    ax.set_title(f'$<\\psi_n, \\psi_m>$ for {titles[i]}')
    plt.imshow(orthonormal_grid, cmap='hot')
    plt.colorbar(orientation='vertical')
plt.show()


From the figures, we observe that the stationary states are orthonormal, for all n and m, and for all potentials. We see that only the diagonal consists of ones, and the rest is zeros. ie, the product is only nonzero (here one) when n = m. 

Nå som vi har bekreftet ortogonaliteten til de stasjonære tilstandene, er vi godt forberedt til å beregne koeffisientene $c_n$ som behøves for å representere en vilkårlig starttilstand $\boldsymbol{\Psi_0}$ som en lineærkombinasjon $\boldsymbol{\Psi_0} = \sum_n c_n \boldsymbol{\psi_n}$ av de stasjonære tilstandene.
For å finne $\boldsymbol{\Psi}$ for $t > 0$, trenger vi så bare sette på eksponentialfaktorene $e^{-i E_n t / \hbar}$.

**Skriv en funksjon som beregner og returnerer koeffisientene $c_n$Â som behøves for å representere en starttilstand $\boldsymbol{\Psi_0}$ som en lineærkombinasjon av de stasjonære tilstandene $\boldsymbol{\psi_n}$ i et diskretisert rom $\boldsymbol{x}$.**

**Skriv også en funksjon som beregner $\boldsymbol{\Psi}$ for en vilkårlig tid $t>0$ ut fra koeffisientene $c_n$, de stasjonære tilstandene $\boldsymbol{\psi_n}$ og energiene $E_n$. Merk at beregningene involverer tallet $i = \sqrt{-1}$, som i Python er representert med `1j`.**

**Du kan godt verifisere at funksjonene i det minste klarer å reprodusere en eller annen starttilstand i et eller annet potensial ved $t = 0$. I de neste oppgavene vil du få testet dem også for $t > 0$.**

In [ ]:
def cn_calc(waves: np.ndarray, initial: np.ndarray, dx: float) -> np.ndarray:  # returns vector with values for c_n
    return waves @ initial * dx


def STDE_calc(t: float, waves: np.ndarray, cn: np.ndarray, E: np.ndarray) -> np.ndarray:  # returns vector with values for
                                                                                          # the time depentent Schrödinger equation
    weighted_waves: np.ndarray = waves * cn[:, np.newaxis]  # [:, np.newaxis] converts cn to a column vector
    time_factor: np.ndarray = np.exp(-1j*E*t/hb)
    return time_factor @ weighted_waves


# calculation of various properties and visualization of results
fig = plt.figure(figsize=(10, 10))
x, dx = np.linspace(-20*a0, +20*a0, 400, retstep=True)  # defines list x and step length dx
for i in range(len(potentials)):
    energies, waves = get_stationary_states(x, potentials[i], m)
    Psi = np.sin(2*x*np.pi/x[-1])
    cn: np.ndarray = cn_calc(waves, Psi, dx)
    STDE: np.ndarray = STDE_calc(0, waves, cn, energies)
    ax = fig.add_subplot(2, 2, i+1)
    ax.set_title(f'STDE og lincomb STDE for {titles[i]}')
    ax.plot(x, Psi, label="Initialbetingelsen $\\Psi_0$")
    ax.plot(x, np.real(STDE), "--", label="$\\Psi_0$ funnet ved cn")
    ax.grid(True)
    ax.legend()
plt.show()


Nå har vi alt vi trenger for å studere tidsutviklingen av bølgefunksjonen.
Dette skal vi gjøre ved å lage en animasjon.
`matplotlib` er ikke et spesielt velegnet bibliotek til dette, men fungerer allikevel godt nok for vårt formål.

Nedenfor har vi satt sammen funksjonen `animate_wave()` som du skal modifisere til å gjøre nettopp dette.
Den tar inn et diskretisert rom $\boldsymbol{x}$, potensialverdier $\boldsymbol{V}$, en masse $m$, en starttilstand $\boldsymbol{\Psi_0}$ og antall bilder som skal vises per sekund, `fps`.
Funksjonen kaller `animate()` én gang per bilde i animasjonen, der en (foreløpig meningsløs) "bølgefunksjon" beregnes og vises i animasjonen.

Nedenfor har vi satt sammen litt mer informasjon om funksjonen og noen tips om hvordan du kan bruke den til å få tilfredsstillende resultater.
* Du kan vise real- eller imaginærdelen til funksjonen som animeres med argumentene `real` og `imag`.
* Du kan bestemme når animasjonen skal starte og slutte med argumentene `t1` og `t2`. Om du ikke ber om annet, vil den begynne ved $t_1 = 0$ og fortsette for evig og alltid til $t_2 = \infty$. Differansen mellom disse tallene bestemmer også varigheten på animasjonen i *sekunder*.
* Du kan bestemme om animasjonen skal vises i sanntid eller forhåndsberegnes med argumentet `realtime`.
    * I sanntidsmodus gjøres beregningene parallelt med animeringen. Du vil se resultater fra første sekund, men animasjonen kan bli hakkete hvis tidssteget er så lite at beregningene ikke klarer å holde følge med animeringen.
    * I forhåndsberegningsmodus bruker maskinen tiden den trenger til å sette sammen alle bildene i animasjonen til en video. Tidssteget vil respekteres, men forhåndsberegningen kan ta en stund og du må bruke et endelig tidsintervall. **Kjør animasjonene i forhåndsberegningsmodus før du leverer oppgaven, slik at de kan spilles av som en video.**

Det er også noen tekniske ting som er greit å merke seg, i tilfelle ting ikke fungerer.
* **Funksjonen `animate_wave()` må kalles nederst i en Jupyter Notebook-celle for at animasjonen skal vises.**
* Hvis du irriterer deg over at figuren som animeres vises to ganger, kan du kommentere ut kodeblokken med `clear_output()`.  **Vær i så fall klar over at dette vil fjerne all output som er generert av en celle fram til `clear_output()` kalles.**
* Sanntidsmodus `realtime=True` kan kun brukes om du har bedt Jupyter Notebook eller Jupyter Lab bruke en interaktiv backend med `%matplotlib notebook` eller `%matplotlib widget` øverst i notebooken. Om ikke må du forhåndsberegne animasjonene med `realtime=False`.

**Sett deg inn i og modifiser funksjonen `animate_wave()`, slik at den animerer $\boldsymbol{\Psi}$ fra starttilstanden $\boldsymbol{\Psi_0}$ i potensialet $V$ i rommet $\boldsymbol{x}$.**

In [ ]:
def animate_wave(x, v, m, wave0, fps, t1=0, t2=None, realtime=True, real=False, imag=False, extra_info: str = ""):
    assert realtime or t2 is not None, "non-realtime animation must be finite in time"
    
    dt = 1 / fps
    nframes = None if t2 is None else int((t2 - t1) / dt)  # None animates forever
    
    fig, ax = plt.subplots()
    
    # prevent duplicate animation figure in non-realtime mode
    if not realtime:
        clear_output()
    
    
    # print information about this animation
    nframesstr = "infinite" if t2 is None else f"{nframes}"
    durationstr = "infinite" if t2 is None else f"{t2-t1:.2f}"
    print("Animation information:")
    print(f"  Frames   : {nframesstr}")
    print(f"  Framerate: {fps} FPS")
    print(f"  Duration : {durationstr}")
    print(f"  Time step: {dt}")
    print(extra_info)
    
    
    ax.set_xlabel("$x$")
    ax.set_ylabel("$|\\Psi|$, $\\Re{(\\Psi)}$, $\\Im{(\\Psi)}$")
    ax.grid(True)
    
    # create objects for the graphs that will be updated every frame
    # the commas matter!
    ymax = max(np.max(np.abs(wave0)**2), np.max(np.abs(wave0)))  # TODO
    ymax = np.max(np.abs(wave0))
    graph, = ax.plot([x[0], x[-1]], [0, +1.5*ymax])  # plot 1.5x wave0 to make room
    if real:
        graph2, = ax.plot([x[0], x[-1]], [0, -1.5*ymax])  # make room for negative values
    if imag:
        graph3, = ax.plot([x[0], x[-1]], [0, -1.5*ymax])  # make room for negative values
    
    # plot potential extended with indications of infinite walls at ends
    ax2 = ax.twinx()
    v_max = np.min(v) + 1.1 * (np.max(v) - np.min(v)) + 1  # + 1 if v = const
    x_ext = np.concatenate(([x[0]], x, [x[-1]]))
    v_ext = np.concatenate(([v_max], v, [v_max]))
    v_color = "black"
    ax2.set_ylabel("$V(x)$", color=v_color)
    ax2.plot(x_ext, v_ext, linewidth=3, color=v_color, label="V")
    ax2.tick_params('y', colors=v_color)
    ax2.legend(loc="upper right")
    
    # call this function for every frame in the animation
    def animate(frame, energies, waves, cn):
        time = t1 + frame*dt
        wave = STDE_calc(time, waves, cn, energies)
        
        # update graph objects
        # set_data() etc. modifies an existing object in a figure
        # it is much more efficient than creating a new figure for every animation frame
        graph.set_data(x, np.abs(wave))
        graph.set_label(f"$|\\Psi(x, t = {time:.2f})|$")
        if real:
            graph2.set_data(x, np.real(wave))
            graph2.set_label(f"$\\Re(\\Psi(x, t = {time:.2f}))$")
        if imag:
            graph3.set_data(x, np.imag(wave))
            graph3.set_label(f"$\\Im(\\Psi(x, t = {time:.2f}))$")     
        ax.legend(loc="upper left")

        

    dx = x[1] - x[0]
    energies, waves = get_stationary_states(x, v, m)
    cn = cn_calc(waves, wave0, dx)
    # create matplotlib animation object
    ani = matplotlib.animation.FuncAnimation(fig, animate, fargs=(energies, waves, cn), frames=nframes, interval=dt*1000, repeat=False) 
    if realtime:
        return ani
    else:
        return HTML(ani.to_jshtml())


I resten av øvingen skal vi rett og slett bare teste ut animasjonsfunksjonen på en del forskjellige situasjoner.

### Partikkel i boks

I regneøvingene har vi sett på superposisjonen av grunntilstanden og første eksiterte tilstand
$$ \Psi(x, 0) = \frac{1}{\sqrt{2}} (\psi_1(x) + \psi_2(x)) $$
for en partikkel i boks.
Vi fant så at sannsynlighetstettheten $|\Psi(x, t)|^2$ skulle oscillere fram og tilbake med perioden
$$ T = \frac{2 \pi \hbar}{E_2 - E_1} $$

**Preparér denne starttilstanden for et elektron i en boks og bekreft at perioden er riktig. Du kan for eksempel variere størrelsen på boksen for å finne en periode du har tid til å observere.**

In [ ]:
x: np.ndarray = np.linspace(0.*a0, 5.*a0, 400)  # x-values, defined as an array
m = me  # the mass of the particle
V_infwell: np.ndarray = np.zeros_like(x)  # initializes potential-vector

    
# calculation of various given properties
energies, waves = get_stationary_states(x, V_infwell, m)
initial: np.ndarray = 1/np.sqrt(2) * (waves[0] + waves[1])
T: float = (2 * np.pi * hb) / (energies[1] - energies[0])

    
# visualization of results
animate_wave(x, V_infwell, m, initial, 20, t2=T+2, realtime=False, real=True, imag=True, extra_info=f"  Period of oscillation: {T:.2f}")


From the frame at $t=0$ and $t=T$, we observe that the wave function is equal. Therefore, T is equal to the analytical period. 

### Fri partikkel representert ved Gaussisk bølgepakke

I regneøvingene har vi også sett hvordan vi kan representere en fri partikkel som en gaussisk bølgepakke
$$ \Psi(x, 0) = (2 \pi \Delta x^2)^{-1/4} e^{-(x-x_0)^2/4 \Delta x^2} e^{i p_0 x / \hbar} $$
med 
$$ \langle x \rangle = x_0 \quad \text{og} \quad \langle p \rangle = p_0 $$
Slik oppnådde vi også et "beste kompromiss" mellom usikkerhetene i posisjon $\Delta x$ og impuls $\Delta p$, nemlig
$$ \Delta x \Delta p = \hbar / 2 $$
Dette er altså det nærmeste vi kommer å kunne representere en fri partikkel med en noenlunde veldefinert posisjon **og** impuls.

**Plassér et elektron i en uendelig brønn med en gaussisk bølgepakke. Sørg for at den har plass til å bevege seg et stykke før den kræsjer i veggen. Studér utviklingen over tid. Hva skjer med formen til bølgepakken over tid? Hva skjer ved veggene? Hvilken hastighet har bølgepakkens tyngdepunkt?**

In [ ]:
def gauss(x: np.ndarray, p0: float, x0: float) -> np.ndarray: # defines the gauss function
    dx = x[1] - x[0]
    return (2*np.pi*dx**2)**(-0.25) * np.exp(-(x-x0)**2 / (4*dx**2)) * np.exp(1j*p0*x/hb)


x, dx = np.linspace(-14*a0, 14*a0, 400, retstep=True)  # x-values, defined as an array. Timestep dx
V_infwell = np.zeros_like(x)  # initializes array for potential
m = me  # the mass of the particle
# expectation value for impulse p and position x
p0 = 15. * hb/a0
x0 = 0. * a0


initial = gauss(x, p0, x0)  # initial gauss function 


# visualization of results
animate_wave(x, V_infwell, m, initial, 25, t1=0, t2=4, realtime=False, real=False, imag=False)


Dispersion takes place over time, and of course, the wave packet collides with the walls and is reflected. This will cause interference to occur.

The speed of the wave package is $v = p_0/m_e$

### Koherent tilstand i harmonisk oscillator

En forskjøvet grunntilstand
$$ \Psi(x, 0) = \left(\frac{m \omega}{\pi \hbar}\right)^{1/4} e^{-m \omega (x-x_0)^2 / \pi \hbar} $$
i den harmoniske oscillatoren kalles på engelsk en *coherent state*.
På norsk kan vi kalle dette en *koherent* eller kanskje en *sammenhengende tilstand*.
Slike tilstander er av spesiell interesse når vi sammenligner kvantemekanikken med den klassiske mekanikken.

**Studér tidsutviklingen til en slik tilstand for et elektron i en harmonisk oscillator. Plassér den et sted til siden for sentrum av oscillatoren. Kan du se for deg motivasjonen bak navngivningen? Hvilken karakteristisk egenskap ved tilstanden tror du gjør den spesielt relevant for sammenligning med klassisk mekanikk?**

In [ ]:
def coherent_state(x: np.ndarray, x0: float, w: float, m: float = me) -> np.ndarray:  # defines the coherent state as given
    return (m*w/(np.pi*hb))**(0.25) * np.exp(-m*w* (x-x0)**2 / (np.pi*hb))


x, dx = np.linspace(-10*a0, 10*a0, 400, retstep=True)  # x-values in an array, and timestep dx


w = 2. * Eh/hb
m = me  # the mass of the particle
x0 = -5. * a0  # start position of the particle
V_harmonic = 0.5 * m * w**2 * x**2  # defines harmonic potential
initial = coherent_state(x, x0, w, m)  # defines the coherent state 

# visualization of results
animate_wave(x, V_harmonic, m, initial, 25, t1=0,t2=10, realtime=False, real=True, imag=True)


Here, no dispersion occurs, and, the amplitude does not decrease with time. This is completely analogous to the block-spring systems in classical mechanics. 

### Transmisjon og refleksjon av fri partikkel ved potensialbarriere

I den klassiske mekanikken kan en partikkel umulig befinne seg i et område der partikkelens energi er lavere enn den potensielle energien.
Men i kvantemekanikken er ikke ting så enkelt.
Her kan en partikkel bevege seg gjennom et potensial, selv om potensialet på et punkt er høyere enn partikkelens energi.
Mulighetene for transmisjon og refleksjon avhenger spesielt av partikkelens (forventede) energi
$$ \langle E \rangle = \sum_{n=1}^N |c_n|^2 E_n $$

**Send et fritt elektron (i form av en gaussisk bølgepakke) inn mot en potensialbarriere. Studér transmisjonen og refleksjonen av bølgepakken. Undersøk hvordan partikkelens (forventede) energi påvirker dens muligheter for å reflekteres og transmitteres, særlig for energier rundt barrierehøyden. Du kan enkelt variere energien ved å justere for eksempel partikkelens impuls.**

 With $p_0 = 0.01~ \hbar / a_0$

In [ ]:
x, dx = np.linspace(-10*a0, 10*a0, 400, retstep=True)  # x-values in an array, and timestep dx
V0 = 100. * Eh  # the potential at the box walls
m = me  # the mass of the particle
# expectation value for impulse p and position x
p0 = 0.01 * hb/a0
x0 = -4. * a0


V_barrier = np.piecewise(x, [np.abs(x) > 1.5*a0, np.abs(x) <= 1.5*a0], [-V0, V0])  # defines the potential barrier
initial = gauss(x, p0, x0)  # initial gauss function 
energies, waves = get_stationary_states(x, V_barrier, m)  # calculates wave functions and energies 
cn = cn_calc(waves, initial, dx)


E: float = np.sum(np.abs(cn)**2*energies)


# visualization of results
animate_wave(x, V_barrier, m, initial, 50, t2=4, realtime=False, real=False, imag=False, extra_info=f"  p0 = {p0:.2f}\n  E0 = {E:.2f}")


 With $p_0 = 20~ \hbar / a_0$

In [ ]:
x, dx = np.linspace(-10*a0, 10*a0, 400, retstep=True)  # x-values in an array, and timestep dx
V0 = 100. * Eh  # the potential at the box walls
m = me  # the mass of the particle
# expectation value for impulse p and position x
p0 = 20. * hb/a0
x0 = -4. * a0


V_barrier = np.piecewise(x, [np.abs(x) > 1.5*a0, np.abs(x) <= 1.5*a0], [-V0, V0])  # defines the potential barrier
initial = gauss(x, p0, x0)  # initial gauss function 
energies, waves = get_stationary_states(x, V_barrier, m)  # calculates wave functions and energies 
cn = cn_calc(waves, initial, dx)


E: float = np.sum(np.abs(cn)**2*energies)


# visualization of results
animate_wave(x, V_barrier, m, initial, 50, t2=4, realtime=False, real=False, imag=False, extra_info=f"  p0 = {p0:.2f}\n  E0 = {E:.2f}")

With $V = 100$, we study different values of $p_0$. With a low $p_0 = 0.01~ \hbar / a_0$, most, if not all of the wave, will be reflected. With $p_0 = 0.01~ \hbar / a_0$, the wave energy is nowhere near the potential. But as we increase the value of $p_0$, the energy increases, and more and more of the wave will be transmitted. As we increase the value of $p_0$, the energy increases. For $p_0 = 20~ \hbar / a_0$, we observe that the energy has exceeded the potential. Now, most of the wave will be transmitted, but we still observe some reflection. 